In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab04.ipynb")

In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import string
import itertools
import re
from functools import reduce

# Lab 4: Discrete Logarithm and Pollard's Rho Algorithm
Contributions From: Ryan Cottone

Welcome to Lab 4! In this lab, we will examine and construct common attacks on asymmetric systems, largely encompassing discrete logarithm and integer factorization. 

## Helpers

In [ ]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modularInverse(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

In [ ]:
def CRT(rList, mList):
    N = 1
    
    for modulo in mList:
        N*=modulo
    
    total = 0
    
    for i in range(len(rList)):
        a_i = rList[i]
        b_i = (N//mList[i])*modularInverse(N//mList[i], mList[i])
        
        total += a_i * b_i
    
    return total % N

## Discrete Logarithm

The **discrete logarithm problem** is the task of finding $x$ from the equation $g^x \equiv h \mod p$. It is considered computationally hard if done properly, and underlies extremely popular cryptosystems like Diffie-Hellman. 

Why can't we just do $\log(g)$ like with normal numbers? Well, it is possible and likely that $g^x > p$, and has therefore wrapped around the modulus. We can't easily reconstruct the number after it's wrapped around, which is why this problem is generally considered hard.

**Question 1**: Implement the trial-and-error algorithm to solve a discrete log problem.

*HINT: Remember to use pow(base, exponent, modulo) to do exponentiation in modular arithmetic!*

In [ ]:
def trialAndErrorDLP(g, h, p):
    # Given g, h, and p, find x using trial and error 
    ...

In [ ]:
grader.check("q1_1")

## Baby Step Giant Step

The Baby Step Giant Step algorithm is an improvement over the trial-and-error algorithm from before. We find some $n$ such that $n = \lceil \sqrt{p} \rceil$, and divide $x$ into $x = qn + r$ for some q, r.

From here, we see that:
$$g^{qn + r} \equiv h \mod p$$
$$g^{r} \equiv h(g^{-qn}) \mod p$$

If we can find $r$ and $q$ such that this equation holds, we can find the discrete log as $x = qn + r$!

First, we make a hashmap **seen** that keeps track of our $g^r$ values. We then do a for loop from 1 to n (inclusive) and store $g^r \mod p$ into **seen** with the key as $r$. This is our "baby step".

Next, we pre-compute $g^{-n}$ as it remains constant. Then, we initialize our val to **h** and do a for-loop from 0 to n-1 (inclusive). At each step, we multiply val by $g^{-n}$ from before. Eventually, we will find an instance that **val** is in **seen**. At this point, we can use **q** and **r** from the **seen** map to return **qn + r**.

**Question 2**: Implement the Baby Step Giant Step algorithm.

In [ ]:
def babyGiant(g, h, p, N=-1):
    if N == -1:
        N = p-1
    
    n = math.ceil(math.sqrt(N))
    
    seen = {}
    
    g_r = 1
    
    # Baby step: calculate g^r for all 0 <= r < n
    # Make sure to store it in the hashmap!
    for r in range(1,n):
        ...
    
    # We can precompute g^(-n) since that remains constant, and just 
    # multiply by val each time for the giant step
    pre = modularInverse(pow(g, n, p), p)
    
    val = h
    
    # Giant step: calculate g^(-qn) for all 0 <= q < n
    # Make sure to compare it with the hashmap!
    for q in range(n):
        ...

In [ ]:
grader.check("q2_1")

## Pollard's Rho

Pollard's Rho algorithm is very efficient at finding small factors of large numbers. It functions by using some iterate function $f(x) = (x^2 + 1) \mod p$ to advance two numbers in a pseudo-random nature. 

At any given point, if $|x - y|$ is a multiple of a factor of $p$, the $\gcd(|x - y|, p)$ will be the shared factor of the two. We repeat our algorithm until we get "lucky" enough for this to happen.

**Question 4**: Implement Pollard's Rho Algorithm.

In [ ]:
def iterate(x, p):
    return (x**2 + 1) % p

def pollardRho(p):
    x = y = 2
    d = 1
    
    # Set x = f(x) and y = f(f(y)), then compute the gcd(|x - y\, p)
    while d == 1:
        ...
    
    return d

In [ ]:
grader.check("q4_1")

Congrats on finishing Lab 4!

## Pohlig-Hellman (Extra Credit)

Pohlig-Hellman is an algorithm for determining the discrete logarithm of a composite modulus. It works by splitting 
the prime factors of $p$ into $p_1, p_2, \ldots, p_k$ (along with their mulitplicities $e_i$).

Given these it, it recursively calls Baby Step Giant Step with $$g_i = g^{\frac{N}{p_i^e}} \mod p$$ and $$h_i = h^{\frac{N}{p_i^e}} \mod p$$.

Given these partial results, it recombines the $x_i$ with the Chinese Remainder Theorem to recover the overall $x$.

In [ ]:
def pohligHellman(g, h, p, orderFactors=[], e=None):
    if len(orderFactors) == 0:
        tempfactors = factor(p-1)
        e = []
        
        for factor in tempfactors:
            orderFactors.append(factor[0])
            e.append(factor[1])
    
    N = 1
    
    if not e:
        e = [1]*len(orderFactors)
    
    for i in range(len(orderFactors)):
        orderFactors[i] = orderFactors[i]**e[i]
        N *= orderFactors[i]
        
    g_arr, h_arr = [], []
    
    
    for i in range(len(orderFactors)):
        ...
    
    crt_arr = []
    
    for i in range(len(g_arr)):
        ...
    
    return CRT(crt_arr, orderFactors) # SOLUTION

In [ ]:
p = 2189248127867
g = 1267362
h = 1244880003213
factors = [2,29,2459,15350003]

assert(pow(g, pohligHellman(g, h, p, factors), p) == h)

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Once you have generated the zip file, go to the Gradescope page for this assignment to submit.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)